In [1]:
# import libraries
import glob
import os,sys, importlib
import types, time, inspect
import pprint
import itertools        
from multiprocessing import Pool

In [2]:
def clear_dir( directory):
    """Remove ../ from the directory address because of sys.path && import_module.
    
    Parameters:
    -----------
    directory:   directory position.
    """
    
    list_path = directory.split('/')
    while '..' in list_path:
        cd = [i for i,x in enumerate(list_path) if x == '..']
        list_path.pop(cd[0])
        list_path.pop(cd[0]-1)
    return '/'.join(list_path)
#print(clear_dir("/home/doren/Desktop/guacamole/test/test_validation/../../src/local/wrapper"))

In [3]:
def find_testable( directory  = '/../src/local/'):
    """Walk through the tree of the src of the project and find:
    1) method
    2) method's position
    3) method's arguments
    
    Parameters:
    ------------
    directory:    where to search related to this position.
    """
    
    wd              = os.getcwd()                 # get working directory
    dir_list        = glob.glob(wd + directory)   # search directories 
  
    list_files      = [] # list of files names
    list_functions  = [] # list of functions
    list_exceptions = [] # list of exceptions from calling names
    # find all files
    for each in dir_list: 
        list_files.append([ name.replace(each,'').replace('.py','') \
                           for name in glob.glob(each + '*.py')])
    # find all arguments for each method in these files
    for each_dir in range(0,len(list_files)):
        sys.path.append(dir_list[each_dir])  # bring the path for importing
        for each in list_files[each_dir]:
            try:   # import class
                x = importlib.import_module(each)
                for method in dir(x):
                    try:
                        method_to_call = getattr(x, method)
                        list_functions.append( { 'position': clear_dir(dir_list[each_dir]+each),
                                                 'method_name': method,
                                                 'arguments': inspect.getfullargspec(method_to_call)})
                    except Exception as e:
                        list_exceptions += [{"name": method, "exception": e}]
            except ImportError as err:
                pass
    return list_functions, list_exceptions

In [4]:
list_functions, list_exceptions = find_testable()
#print("\nMethods\n")
#pprint.pprint(list_functions)
#print("\nExceptions\n")
#pprint.pprint(list_exceptions)

In [5]:
def get_extra( list_exceptions):
    """Find libraries and classes imported.
    
    Parameters:
    -------------
    list_exceptions:  result from testable files"""
    
    libraries =[]
    ## find libraries in a src code tree
    for each in list_exceptions:
        if not each['name'].startswith('__'):
            libraries.append(each['name'])
    libraries = list(set(libraries))
    return libraries
#print(get_extra(list_exceptions))

In [6]:
input_trials = {
    "none": [None],
    "boolean": [True, False],
    "numbers": [-1,0, 1.1, 100, 1.0j ],
    "string":  ["testing", u"testing"],
    "tuple": [("test_1",0), (None,0 ), ("test", None)],
    "list": [["test_1",0],[None,0],["test_1", None],[],[0]],
    "dict": [{"test_1":0},{None:0},{"test_1":None}]
}
      
list_trials = [] # all the options you want to try
for each in input_trials.keys():
    list_trials += input_trials[each]
#print(len(list_trials))
list_trials = list_trials[5:9]

In [7]:
def caller(func, args):
    """Used to call the methods by passing a list args.
    Parameters:
    ------------
    func:   the method to be called.
    args:   the arguments to be passed.
    """
    
    func(*args)

def generate_trials( list_trials , num_args):
    """Generate input for the tests depending on the number of args.
    
    Parameters:
    ------------
    list_input:    all possible input listed for generation
    num_args:      number of arguments the method needs to take    
    """
    
    return [list(p) for p in itertools.product(list_trials, repeat=num_args)]

In [8]:
def test_verification( list_functions = [], verbose = False):
    """Tests if the methods crash because of wrong input.
    
    Parameters:
    -----------
    list_functions:   input methods to check if they run properly with different input
    """
    
    # for each method get the number of arguments
    for each in list_functions:
        each['acceptable_input'] = []
        try:       # import the file and methods
            last_sep = each['position'].rfind('/')
            sys.path.append( each['position'][ : last_sep])
            env = importlib.import_module( each['position'][ last_sep+1 : ])
        except Exception as e:
            print(e)
            continue
        method_to_call = getattr( env, each[ 'method_name']) # callable
        args = each['arguments']   # get args
        if args[0][0] == 'self':   # class: needs other testing
            continue 
        try:                       # simple testing
            err_ar = (len(args[0])+ 1)*[0]
            caller( method_to_call, err_ar)
            each['acceptable_input'].append(err_ar)
        except Exception as e:
            None
        list_input = generate_trials(list_trials, len(args[0]))
        for each_input in list_input: # try for each input type
            if verbose:
                print('Trying: ', each_input)
            try:                      # save input as acceptable
                caller( method_to_call, each_input)
                each['acceptable_input'].append( each_input)
            except Exception as e:    # skip
                None
    return list_functions

In [9]:
test_verification( list_functions[:2], True)

Trying:  [1.1, 1.1]
Trying:  [1.1, 100]
Trying:  [1.1, 1j]
Trying:  [1.1, 'testing']
Trying:  [100, 1.1]
Trying:  [100, 100]
Trying:  [100, 1j]
Trying:  [100, 'testing']
Trying:  [1j, 1.1]
Trying:  [1j, 100]
Trying:  [1j, 1j]
Trying:  [1j, 'testing']
Trying:  ['testing', 1.1]
Trying:  ['testing', 100]
Trying:  ['testing', 1j]
Trying:  ['testing', 'testing']


[{'position': '/home/doren/Desktop/guacamole/src/local/wrapper',
  'method_name': 'Spinner',
  'arguments': FullArgSpec(args=['self', 'delay'], varargs=None, varkw=None, defaults=(None,), kwonlyargs=[], kwonlydefaults=None, annotations={}),
  'acceptable_input': []},
 {'position': '/home/doren/Desktop/guacamole/src/local/wrapper',
  'method_name': 'adder',
  'arguments': FullArgSpec(args=['a', 'b'], varargs=None, varkw=None, defaults=(4, 5), kwonlyargs=[], kwonlydefaults=None, annotations={}),
  'acceptable_input': [['testing', 'testing']]}]

In [10]:
pprint.pprint(list_functions[10:20])

[{'arguments': FullArgSpec(args=['url', 'output'], varargs=None, varkw=None, defaults=('subdomains.txt',), kwonlyargs=[], kwonlydefaults=None, annotations={}),
  'method_name': 'call_subdomains',
  'position': '/home/doren/Desktop/guacamole/src/local/tools'},
 {'arguments': FullArgSpec(args=['website'], varargs=None, varkw=None, defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={}),
  'method_name': 'check_dmarc',
  'position': '/home/doren/Desktop/guacamole/src/local/tools'},
 {'arguments': FullArgSpec(args=['target'], varargs=None, varkw=None, defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={}),
  'method_name': 'clear_url',
  'position': '/home/doren/Desktop/guacamole/src/local/tools'},
 {'arguments': FullArgSpec(args=['url'], varargs=None, varkw=None, defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={}),
  'method_name': 'does_url_exist',
  'position': '/home/doren/Desktop/guacamole/src/local/tools'},
 {'arguments': FullArgSpec(args=['store

In [11]:
def test_validation( dict_input = {}, dict_output = {}):
    """Tests if results are equal to expected.
    
    Parameters:
    ------------
    dict_input:   input methods
    dict_output:  expected output of the input methods found in file
    """

In [12]:
#importlib_suffixes.py
# supported types for the current platform and the parameters
# for loading them
import importlib.machinery

SUFFIXES = [
    ('Source:', importlib.machinery.SOURCE_SUFFIXES),
    ('Debug:',
     importlib.machinery.DEBUG_BYTECODE_SUFFIXES),
    ('Optimized:',
     importlib.machinery.OPTIMIZED_BYTECODE_SUFFIXES),
    ('Bytecode:', importlib.machinery.BYTECODE_SUFFIXES),
    ('Extension:', importlib.machinery.EXTENSION_SUFFIXES),
]


def main():
    tmpl = '{:<10}  {}'
    for name, value in SUFFIXES:
        print(tmpl.format(name, value))


if __name__ == '__main__':
    main()

Source:     ['.py']
Debug:      ['.pyc']
Optimized:  ['.pyc']
Bytecode:   ['.pyc']
Extension:  ['.cpython-37m-x86_64-linux-gnu.so', '.abi3.so', '.so']
